In [1]:
from datetime import datetime, timedelta
import sqlite3

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass

from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest, StockBarsRequest
from alpaca.data.timeframe import TimeFrame

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, TrailingStopOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderType
import numpy as np

from polygon import ReferenceClient
import pandas as pd


In [2]:


class LiveTrader:
    def __init__(self, api_key, secret_key, polygon_key):
        self.trading_client = TradingClient(api_key, secret_key)
        self.stock_client = StockHistoricalDataClient(api_key, secret_key)
        self.polygon_client = ReferenceClient(polygon_key)
        self._init_db()  # Initialize database

    def _init_db(self):
        self.conn = sqlite3.connect('trading_data.db')
        cursor = self.conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS framestack (
                          id INTEGER PRIMARY KEY AUTOINCREMENT,
                          obs BLOB)''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS account_info (
                          id INTEGER PRIMARY KEY AUTOINCREMENT,
                          balance REAL,
                          trades TEXT)''')
        self.conn.commit()

    def convert_cyrpto_ticker_symbol(self, symbol):
        if 'X:' in symbol:
            # Remove the 'X:' prefix and replace 'USD' with '/USD'
            return symbol.replace('X:', '')
        else:
            return symbol

    def buy(self, symbol, qty, price=None):
        symbol = self.convert_cyrpto_ticker_symbol(symbol)

        print(f"Buying {qty:.2f} shares of {symbol} at {price:.2f}")
        # preparing orders
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC
        )

        # Market order
        return self.trading_client.submit_order(
            order_data=market_order_data
        )

    def sell(self, symbol, qty, price=None, liquidate=False):
        symbol = self.convert_cyrpto_ticker_symbol(symbol)

        print(f"Selling {qty:.2f} shares of {symbol} at {price:.2f}")

        if liquidate:

            print(f'{symbol} being fully liquidated')

            pos = self.get_all_positions()

            qty = float(pos[symbol]['qty'])

            # preparing orders
            market_order_data = MarketOrderRequest(
                symbol=symbol,
                qty=qty,
                side=OrderSide.SELL,
                time_in_force=TimeInForce.GTC
            )

            # Market order
            return self.trading_client.submit_order(
                order_data=market_order_data
            )
        else:
            # preparing orders
            market_order_data = MarketOrderRequest(
                symbol=symbol,
                qty=qty,
                side=OrderSide.SELL,
                time_in_force=TimeInForce.GTC
            )

            # Market order
            return self.trading_client.submit_order(
                order_data=market_order_data
            )

    def get_all_positions(self):
        return self.trading_client.get_all_positions()

    def get_account(self):
        return self.trading_client.get_account()


In [3]:
import os
from dotenv import load_dotenv

In [6]:
# Load environment variables from .env file
load_dotenv('../keys.env')


True

In [7]:
# API keys
api_key = os.environ.get("APCA_API_KEY_ID")
secret_key = os.environ.get("APCA_API_SECRET_KEY")
polygon_key = os.environ.get("POLYGON_API_KEY")

In [8]:
# Initialize LiveTrader
live_trader = LiveTrader(api_key, secret_key, polygon_key)

In [10]:
x = live_trader.get_all_positions()

In [22]:
# iterate a list of dictionaries and find the one with the matching key value
for i in x:
    if i.symbol == 'BCHUSD':
        print(i.qty_available)

10.345040253
